In [8]:
ch_host = "10.120.1.11"
ch_port = 8123
ch_username = "amarbuliev"
ch_password = "KuUNgVlPQiSN6dRqk"

In [9]:
from footprint_dftools.clickhouse import clickhouse_connection as ch

connection = ch.ClickHouseConnection(ch_host, ch_username, ch_password)

In [16]:
connection.read_database('TRUNCATE TABLE sandbox.ongoing_projects')

""


In [17]:
connection.read_database('SELECT * FROM sandbox.ongoing_projects')

,id,status,statusDesc,nameRus,head,directionHead,type,typeDesc,typeId,statusId,...,payed,projectOfficeControl,studentsCount,searchString,clientLogo,detailed_team,projectIndustryLabel,leaders,initiatorId,thumbnail
